In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(3)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [5]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   882 ps 	/   264.6 mm
Bob_T   :    29 ps 	/     8.7 mm
Bob_R   :  -589 ps 	/  -176.7 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    38 ps 	/    11.4 mm
Bob_T   :     0 ps 	/     0.0 mm
Bob_R   :     0 ps 	/     0.0 mm


In [66]:
KMC.rotate_simulataneously(0, 0)

In [65]:
KMC.rotate_simulataneously(0, 45)

In [24]:
n_values = 300
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

In [12]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [11]:

KMC.rotate_simulataneously(0, 45, wait_for_completion=False)

In [10]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [14]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, 11.25, -11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9, debug=True)



corr[a,b] = 0.70757
	N[|T,T>]=   969	(0.316)
	N[|T,R>]=   253	(0.083)
	N[|R,T>]=   195	(0.064)
	N[|R,R>]=  1647	(0.538)

corr[a,B] = 0.7257
	N[|T,T>]=   984	(0.322)
	N[|T,R>]=   210	(0.069)
	N[|R,T>]=   209	(0.068)
	N[|R,R>]=  1652	(0.541)

corr[A,b] = 0.54993
	N[|T,T>]=   690	(0.239)
	N[|T,R>]=   204	(0.071)
	N[|R,T>]=   445	(0.154)
	N[|R,R>]=  1545	(0.536)

corr[A,B] = -0.41586
	N[|T,T>]=   290	(0.117)
	N[|T,R>]=   729	(0.295)
	N[|R,T>]=  1021	(0.413)
	N[|R,R>]=   432	(0.175)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.3990556396317024


In [28]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.691 -0.675 -0.586  0.601]
	N[a , b ]=[1026  237  209 1633]
	N[a , b ]=[ 189 1256  947  361]
	N[a , b ]=[  90  663 1054  516]
	N[a , b ]=[ 501  136  314 1883]

corr[a,B] = [ 0.664 -0.676 -0.699  0.695]
	N[a , B ]=[1002  221  189 1650]
	N[a , B ]=[ 167 1336  961  336]
	N[a , B ]=[ 127  678 1172  352]
	N[a , B ]=[ 456  169  189 2169]

corr[A,b] = [ 0.461 -0.415 -0.449  0.398]
	N[A , b ]=[ 744  195  376 1404]
	N[A , b ]=[157 968 866 656]
	N[A , b ]=[346 766 924 636]
	N[A , b ]=[ 621  522  304 1595]

corr[A,B] = [-0.561  0.546  0.486 -0.45 ]
	N[A , B ]=[ 277  752 1011  477]
	N[A , B ]=[ 570  370  290 1584]
	N[A , B ]=[ 839  141  365 1494]
	N[A , B ]=[119 990 885 690]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.3780884  2.31137612 2.22037178 2.14341162]


In [97]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
TTC.measureS(CHSH_angles=johannes_classical_angles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiMinus, coincidence_window_SI=0.5e-9)


corr[a,b] = -1.0
	N[|T,T>]=     0	(0.000)
	N[|T,R>]=  2539	(0.508)
	N[|R,T>]=  2461	(0.492)
	N[|R,R>]=     0	(0.000)

corr[a,B] = 0.0136
	N[|T,T>]=  1277	(0.255)
	N[|T,R>]=  1234	(0.247)
	N[|R,T>]=  1232	(0.246)
	N[|R,R>]=  1257	(0.251)

corr[A,b] = -0.0068
	N[|T,T>]=  1249	(0.250)
	N[|T,R>]=  1258	(0.252)
	N[|R,T>]=  1259	(0.252)
	N[|R,R>]=  1234	(0.247)

corr[A,B] = 1.0
	N[|T,T>]=  2485	(0.497)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=  2515	(0.503)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 1.9932


# OSC Server Setup


In [7]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('C_diagonal',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('C_diagonal',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('C_diagonal',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('C_diagonal',)
Received OSC_message: ('Q_diagonal',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_

In [8]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [9]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [19]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, -11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [20]:
Myself.client.send_message("/bruQner/set/state", 'C_single')

State Changed to: C_single
Received OSC_message: ('Q_all',)


In [ ]:
stop_performance()

performance stopped...


In [23]:
start_performance()

Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 2, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [2, 2, 1, 1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 2, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [1, 2, 1, 1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sen